In [1]:
%pip install ../..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=49799 sha256=c70928d02a5854df9015b8a35e038ba7d6c4f1d36f791eb39a12c2bde29a9c22
  Stored in directory: /tmp/pip-ephem-wheel-cache-lidigncd/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    train=True
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    train=False
)
testset.initialize()


In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.group_inference import JTTInference
from spuco.models import model_factory 
from spuco.utils import Trainer
from torch.optim import SGD

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
trainer.train(1)

Epoch 0: 100%|██████████| 938/938 [00:03<00:00, 260.58batch/s, accuracy=100.0%, loss=0.00379] 


In [6]:
from spuco.utils import get_labels 

predictions = torch.argmax(trainer.get_trainset_outputs(), dim=-1).detach().cpu().tolist()
jtt = JTTInference(
    predictions=predictions,
    labels=get_labels(trainset)
)

Getting Trainset Outputs: 100%|██████████| 938/938 [00:00<00:00, 1146.86batch/s]


In [7]:
group_partition = jtt.infer_groups()

In [8]:
for key in group_partition.keys():
    print(key, len(group_partition[key]))

(0, 0) 59669
(0, 1) 331


In [9]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 100.0
Group (0, 1) Test Accuracy: 0.0


{(0, 0): 100.0, (0, 1): 0.0}

In [10]:
from spuco.utils import GroupLabeledDataset

group_labeled_trainset = GroupLabeledDataset(trainset, group_partition)

In [11]:
from torch.optim import SGD
from spuco.invariant_train import CustomSampleERM
from spuco.models import model_factory 

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

jtt_train = CustomSampleERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    indices=group_partition[(0,0)] + 200 * group_partition[(0,1)],
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
jtt_train.train()

Epoch 0: 100%|██████████| 1967/1967 [00:06<00:00, 313.81batch/s, accuracy=77.77777777777777%, loss=0.473]


In [12]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 98.81796690307328
Group (0, 1) Test Accuracy: 69.73995271867612
Group (0, 2) Test Accuracy: 60.7565011820331
Group (0, 3) Test Accuracy: 76.59574468085107
Group (0, 4) Test Accuracy: 63.829787234042556
Group (1, 0) Test Accuracy: 70.6601466992665
Group (1, 1) Test Accuracy: 92.42053789731051
Group (1, 2) Test Accuracy: 60.049019607843135
Group (1, 3) Test Accuracy: 36.27450980392157
Group (1, 4) Test Accuracy: 36.76470588235294
Group (2, 0) Test Accuracy: 60.266666666666666
Group (2, 1) Test Accuracy: 75.2
Group (2, 2) Test Accuracy: 96.26666666666667
Group (2, 3) Test Accuracy: 55.2
Group (2, 4) Test Accuracy: 12.566844919786096
Group (3, 0) Test Accuracy: 47.98994974874372
Group (3, 1) Test Accuracy: 33.50125944584383
Group (3, 2) Test Accuracy: 53.904282115869016
Group (3, 3) Test Accuracy: 95.71788413098237
Group (3, 4) Test Accuracy: 77.58186397984886
Group (4, 0) Test Accuracy: 55.415617128463474
Group (4, 1) Test Accuracy: 16.876574307304786
Group (4,

{(0, 0): 98.81796690307328,
 (0, 1): 69.73995271867612,
 (0, 2): 60.7565011820331,
 (0, 3): 76.59574468085107,
 (0, 4): 63.829787234042556,
 (1, 0): 70.6601466992665,
 (1, 1): 92.42053789731051,
 (1, 2): 60.049019607843135,
 (1, 3): 36.27450980392157,
 (1, 4): 36.76470588235294,
 (2, 0): 60.266666666666666,
 (2, 1): 75.2,
 (2, 2): 96.26666666666667,
 (2, 3): 55.2,
 (2, 4): 12.566844919786096,
 (3, 0): 47.98994974874372,
 (3, 1): 33.50125944584383,
 (3, 2): 53.904282115869016,
 (3, 3): 95.71788413098237,
 (3, 4): 77.58186397984886,
 (4, 0): 55.415617128463474,
 (4, 1): 16.876574307304786,
 (4, 2): 18.89168765743073,
 (4, 3): 61.111111111111114,
 (4, 4): 92.92929292929293}

In [13]:
evaluator.worst_group_accuracy

((2, 4), 12.566844919786096)

In [14]:
evaluator.average_accuracy

95.03455987829123

In [15]:
evaluator.evaluate_spurious_task()

43.27